In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow_names.csv')

In [4]:
dataset.head(10)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0.0,0.0,NaN,1.488

In [5]:
dataset.user.unique().tolist() #all users

['Luca',
 'andrea',
 'Federica',
 'michelangelo',
 'Damiano',
 'Claudio',
 'Vincenzo',
 'Serena',
 'Pierpaolo',
 'IvanHeibi',
 'AndreaCarpineti',
 'Elena',
 'Riccardo']

In [6]:
for name, group in dataset.groupby('user'):
    print(name,len(group)) 

AndreaCarpineti 227
Claudio 90
Damiano 531
Elena 79
Federica 214
IvanHeibi 271
Luca 820
Pierpaolo 331
Riccardo 17
Serena 167
Vincenzo 279
andrea 2470
michelangelo 397


In [7]:
dataset.target.unique().tolist() #all users

['Still', 'Car', 'Train', 'Bus', 'Walking']

In [31]:
for name, group in dataset.groupby('target'):
    print(name,len(group)) 

Bus 1178
Car 1180
Still 1179
Train 1179
Walking 1177


In [8]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [32]:
#dataset.columns

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [33]:
#dataset.info()

In [11]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.146752,14.137338,0.081745,15.141369,15.34

In [12]:
dataset.isnull().sum()

id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                       720
android.sensor.gyroscope#min                    

In [13]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-13-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.00

In [14]:
dataset['rv_gyro_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']
dataset['lin_speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']


In [15]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'rv_gyro_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'lin_speed_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'user',
                    'target'
                 ])]

In [35]:
dataset.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'rv_gyro_mean',
       'gyroscope_std', 'linear_acceleration_mean', 'lin_speed_mean',
       'rotation_vector_mean', 'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'user', 'target'],
      dtype='object')

In [16]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,rv_gyro_mean,gyroscope_std,linear_acceleration_mean,lin_speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.000083,0.000737,0.020978,0.000000,0.050413,0.056351,NaN,51.199707,NaN,Luca,Still
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.036325,0.020290,0.879220,14.541729,0.999981,0.999999,NaN,82.409890,89.065143,Luca,Car
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.000931,0.000713,0.034110,0.000000,0.610456,0.610456,NaN,55.501802,NaN,Luca,Still
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.036372,0.010553,1.488361,26.403360,0.998112,0.998112,NaN,95.664309,87.470377,Luca,Car
4,8.965621,10.891645,0.504117,0.183202,0.265652,0.059035,0.098819,1.658308,14.924768,0.322242,0.378193,NaN,156.795909,89.770732,andrea,Car


In [17]:
# split user = ['Luca' 'Pierpaolo' 'IvanHeibi']
dataset_test = dataset[(dataset['user'] == 'Luca')|(dataset['user'] == 'Pierpaolo')|(dataset['user'] == 'IvanHeibi')]
dataset_train = dataset[(dataset['user'] != 'Luca')&(dataset['user'] != 'Pierpaolo')&(dataset['user'] != 'IvanHeibi')]
print(len(dataset))
print(len(dataset_test))
print(len(dataset_train))

5893
1422
4471


In [18]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,rv_gyro_mean,gyroscope_std,linear_acceleration_mean,lin_speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
4487,8.558131,11.201494,0.668099,0.022604,0.034770,0.007781,0.011856,0.402482,12.879413,0.344225,0.350810,NaN,164.029530,NaN,andrea,Car
3908,6.945241,15.460978,1.405397,0.623755,0.138811,0.433605,0.025362,1.395611,18.071424,0.695152,0.695152,NaN,171.175086,87.353453,michelangelo,Bus
3552,10.183943,10.221213,0.011113,0.006450,0.035292,0.006432,0.003236,0.026270,NaN,0.997088,0.997171,NaN,100.595799,82.668862,andrea,Still
5691,9.835555,10.135506,0.090694,0.019763,0.042137,0.019337,0.009980,0.161131,NaN,0.978425,0.978543,NaN,116.862215,89.724358,andrea,Train
1704,9.385085,10.630317,0.273996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.524724,Federica,Car
2896,9.719546,10.983162,0.347928,0.020648,0.063712,0.007302,0.010120,0.144082,3.746132,0.353646,0.373211,NaN,150.006376,89.766219,andrea,Car
1159,9.514564,9.738298,0.067834,0.009225,0.044069,0.003409,0.003055,0.111269,NaN,0.369539,0.372738,NaN,132.750125,89.284722,andrea,Train
5539,8.542446,11.228146,0.518281,0.036119,0.092194,0.023708,0.044557,0.722758,8.749952,0.656397,0.665346,NaN,166.421423,81.796690,michelangelo,Bus
4586,9.827059,9.874919,0.010491,0.001922,0.049631,0.000183,0.000275,0.035393,0.006961,0.095442,0.095442,NaN,279.421291,33.064250,Damiano,Still
845,9.345883,9.802505,0.140469,0.026871,0.036118,0.016478,0.017675,0.176540,1.191648,0.613212,0.625093,NaN,148.897315,89.759162,andrea,Bus


In [19]:
#dataset_train = dataset_train.fillna(0)

In [20]:
dataset_train.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,rv_gyro_mean,gyroscope_std,linear_acceleration_mean,lin_speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
4,8.965621,10.891645,0.504117,0.183202,0.265652,0.059035,0.098819,1.658308,14.924768,0.322242,0.378193,NaN,156.795909,89.770732,andrea,Car
5,9.149029,9.520044,0.103857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.361127,Federica,Car
6,9.758751,9.816197,0.013489,0.004100,0.042991,0.000825,0.001801,0.027441,NaN,0.201198,0.201318,NaN,170.521377,65.158369,michelangelo,Train
8,8.996134,10.027802,0.255053,0.041554,0.065754,0.032445,0.030074,0.278997,1.325235,0.780795,0.786845,NaN,134.611517,89.808441,andrea,Bus
9,9.014786,10.068528,0.267791,0.037451,0.030491,0.036255,0.021391,0.255172,NaN,0.968082,0.968806,NaN,140.174223,89.815738,andrea,Train


In [21]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [22]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            'rv_gyro_mean',
            'linear_acceleration_mean',
            'lin_speed_mean',
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min'
            ]


In [23]:
# YOUR CODE HERE
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(missing_values=np.nan ,strategy='mean')),
  ('Normalizer',RobustScaler())
  
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'rv_gyro_mean',
                                  'linear_acceleration_mean', 'lin_speed_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min'])])

In [24]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [25]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('Normalizer',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'rv_gyro_mean',
                                                   'linear_acceleration_mean',
                                                   'lin_speed_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [26]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_test = dataset_test.drop(['target'], axis=1)
y_test = dataset_test.target

In [36]:
x_train.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'rv_gyro_mean',
       'gyroscope_std', 'linear_acceleration_mean', 'lin_speed_mean',
       'rotation_vector_mean', 'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min'],
      dtype='object')

In [27]:
print(x_train.head())
#print(x_test.head(1))
print(y_train.head())
#print(y_test.head(1))

   accelerometer_min  accelerometer_max  accelerometer_std  gyroscope_mean  \
4           8.965621          10.891645           0.504117        0.183202   
5           9.149029           9.520044           0.103857             NaN   
6           9.758751           9.816197           0.013489        0.004100   
8           8.996134          10.027802           0.255053        0.041554   
9           9.014786          10.068528           0.267791        0.037451   

   gyroscope_uncalibrated_mean  rv_gyro_mean  gyroscope_std  \
4                     0.265652      0.059035       0.098819   
5                          NaN           NaN            NaN   
6                     0.042991      0.000825       0.001801   
8                     0.065754      0.032445       0.030074   
9                     0.030491      0.036255       0.021391   

   linear_acceleration_mean  lin_speed_mean  rotation_vector_mean  \
4                  1.658308       14.924768              0.322242   
5             

In [28]:
from sklearn import model_selection
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_train, y_train,
    test_size=0.2,
    stratify = y_train,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:55:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.08444
0:	learn: 1.4921937	total: 89.4ms	remaining: 1m 29s
1:	learn: 1.3947757	total: 115ms	remaining: 57.4s
2:	learn: 1.3054357	total: 141ms	remaining: 46.8s
3:	learn: 1.2285895	total: 170ms	remaining: 42.3s
4:	learn: 1.1654832	total: 188ms	remaining: 37.5s
5:	learn: 1.1112658	total: 217ms	remaining: 36s
6:	learn: 1.0604803	total: 249ms	remaining: 35.3s
7:	learn: 1.0174841	total: 265ms	remaining: 32.8s
8:	learn: 0.9789992	total: 288ms	remaining: 31.7s
9:	learn: 0.9402964	total: 315ms	remaining: 31.2s
10:	learn: 0.9079710	total: 343ms	remaining: 30.9s
11:	learn: 0.8796478	total: 370ms	remaining: 30.5s
12:	learn: 0.8555647	total: 390ms	remaining: 29.6s
13:	learn: 0.8317506	total: 412ms	remaining: 29s
14:	learn: 0.807

155:	learn: 0.2404842	total: 4.66s	remaining: 25.2s
156:	learn: 0.2390315	total: 4.69s	remaining: 25.2s
157:	learn: 0.2384949	total: 4.72s	remaining: 25.1s
158:	learn: 0.2378569	total: 4.75s	remaining: 25.1s
159:	learn: 0.2366972	total: 4.78s	remaining: 25.1s
160:	learn: 0.2352794	total: 4.8s	remaining: 25s
161:	learn: 0.2347045	total: 4.84s	remaining: 25s
162:	learn: 0.2341202	total: 4.87s	remaining: 25s
163:	learn: 0.2329749	total: 4.91s	remaining: 25s
164:	learn: 0.2322434	total: 4.95s	remaining: 25s
165:	learn: 0.2313164	total: 4.97s	remaining: 25s
166:	learn: 0.2307188	total: 5.01s	remaining: 25s
167:	learn: 0.2300247	total: 5.04s	remaining: 25s
168:	learn: 0.2289269	total: 5.07s	remaining: 24.9s
169:	learn: 0.2282764	total: 5.09s	remaining: 24.8s
170:	learn: 0.2275121	total: 5.14s	remaining: 24.9s
171:	learn: 0.2265279	total: 5.17s	remaining: 24.9s
172:	learn: 0.2259144	total: 5.19s	remaining: 24.8s
173:	learn: 0.2252402	total: 5.23s	remaining: 24.8s
174:	learn: 0.2246628	total: 

317:	learn: 0.1538238	total: 9.85s	remaining: 21.1s
318:	learn: 0.1535178	total: 9.88s	remaining: 21.1s
319:	learn: 0.1532833	total: 9.92s	remaining: 21.1s
320:	learn: 0.1528303	total: 9.95s	remaining: 21s
321:	learn: 0.1524703	total: 9.98s	remaining: 21s
322:	learn: 0.1520409	total: 10s	remaining: 21s
323:	learn: 0.1518164	total: 10s	remaining: 21s
324:	learn: 0.1511911	total: 10.1s	remaining: 20.9s
325:	learn: 0.1509567	total: 10.1s	remaining: 20.9s
326:	learn: 0.1506930	total: 10.1s	remaining: 20.9s
327:	learn: 0.1503456	total: 10.2s	remaining: 20.8s
328:	learn: 0.1502246	total: 10.2s	remaining: 20.8s
329:	learn: 0.1497741	total: 10.2s	remaining: 20.7s
330:	learn: 0.1494673	total: 10.3s	remaining: 20.7s
331:	learn: 0.1492642	total: 10.3s	remaining: 20.7s
332:	learn: 0.1489963	total: 10.3s	remaining: 20.7s
333:	learn: 0.1485890	total: 10.4s	remaining: 20.7s
334:	learn: 0.1482773	total: 10.4s	remaining: 20.7s
335:	learn: 0.1479798	total: 10.4s	remaining: 20.6s
336:	learn: 0.1476325	to

480:	learn: 0.1131540	total: 15.1s	remaining: 16.3s
481:	learn: 0.1130712	total: 15.2s	remaining: 16.3s
482:	learn: 0.1128947	total: 15.2s	remaining: 16.3s
483:	learn: 0.1127257	total: 15.3s	remaining: 16.3s
484:	learn: 0.1125172	total: 15.3s	remaining: 16.2s
485:	learn: 0.1123598	total: 15.3s	remaining: 16.2s
486:	learn: 0.1122253	total: 15.3s	remaining: 16.2s
487:	learn: 0.1120622	total: 15.4s	remaining: 16.1s
488:	learn: 0.1119320	total: 15.4s	remaining: 16.1s
489:	learn: 0.1118086	total: 15.4s	remaining: 16.1s
490:	learn: 0.1116557	total: 15.5s	remaining: 16s
491:	learn: 0.1114608	total: 15.5s	remaining: 16s
492:	learn: 0.1113156	total: 15.6s	remaining: 16s
493:	learn: 0.1110610	total: 15.6s	remaining: 16s
494:	learn: 0.1108899	total: 15.6s	remaining: 15.9s
495:	learn: 0.1107798	total: 15.7s	remaining: 15.9s
496:	learn: 0.1105813	total: 15.7s	remaining: 15.9s
497:	learn: 0.1103357	total: 15.7s	remaining: 15.9s
498:	learn: 0.1101203	total: 15.7s	remaining: 15.8s
499:	learn: 0.110004

644:	learn: 0.0888080	total: 20.7s	remaining: 11.4s
645:	learn: 0.0886608	total: 20.7s	remaining: 11.3s
646:	learn: 0.0885362	total: 20.7s	remaining: 11.3s
647:	learn: 0.0883725	total: 20.8s	remaining: 11.3s
648:	learn: 0.0882063	total: 20.8s	remaining: 11.3s
649:	learn: 0.0880992	total: 20.8s	remaining: 11.2s
650:	learn: 0.0879884	total: 20.9s	remaining: 11.2s
651:	learn: 0.0878161	total: 20.9s	remaining: 11.2s
652:	learn: 0.0876250	total: 21s	remaining: 11.2s
653:	learn: 0.0875332	total: 21s	remaining: 11.1s
654:	learn: 0.0874290	total: 21.1s	remaining: 11.1s
655:	learn: 0.0873298	total: 21.1s	remaining: 11.1s
656:	learn: 0.0871820	total: 21.1s	remaining: 11s
657:	learn: 0.0871166	total: 21.2s	remaining: 11s
658:	learn: 0.0869751	total: 21.3s	remaining: 11s
659:	learn: 0.0869111	total: 21.3s	remaining: 11s
660:	learn: 0.0867687	total: 21.3s	remaining: 10.9s
661:	learn: 0.0866600	total: 21.4s	remaining: 10.9s
662:	learn: 0.0865034	total: 21.4s	remaining: 10.9s
663:	learn: 0.0864222	to

806:	learn: 0.0723852	total: 26.1s	remaining: 6.25s
807:	learn: 0.0723133	total: 26.2s	remaining: 6.21s
808:	learn: 0.0721982	total: 26.2s	remaining: 6.18s
809:	learn: 0.0721253	total: 26.2s	remaining: 6.15s
810:	learn: 0.0720666	total: 26.3s	remaining: 6.12s
811:	learn: 0.0719677	total: 26.3s	remaining: 6.08s
812:	learn: 0.0718949	total: 26.3s	remaining: 6.05s
813:	learn: 0.0718413	total: 26.4s	remaining: 6.02s
814:	learn: 0.0717716	total: 26.4s	remaining: 5.99s
815:	learn: 0.0716727	total: 26.4s	remaining: 5.96s
816:	learn: 0.0716266	total: 26.5s	remaining: 5.94s
817:	learn: 0.0715460	total: 26.5s	remaining: 5.9s
818:	learn: 0.0714582	total: 26.6s	remaining: 5.87s
819:	learn: 0.0713763	total: 26.6s	remaining: 5.84s
820:	learn: 0.0712656	total: 26.6s	remaining: 5.81s
821:	learn: 0.0712003	total: 26.7s	remaining: 5.77s
822:	learn: 0.0710728	total: 26.7s	remaining: 5.74s
823:	learn: 0.0709972	total: 26.7s	remaining: 5.71s
824:	learn: 0.0709314	total: 26.7s	remaining: 5.67s
825:	learn: 0

970:	learn: 0.0600398	total: 31.5s	remaining: 942ms
971:	learn: 0.0599970	total: 31.6s	remaining: 909ms
972:	learn: 0.0598675	total: 31.6s	remaining: 876ms
973:	learn: 0.0598455	total: 31.6s	remaining: 844ms
974:	learn: 0.0597628	total: 31.6s	remaining: 811ms
975:	learn: 0.0597212	total: 31.7s	remaining: 778ms
976:	learn: 0.0596218	total: 31.7s	remaining: 746ms
977:	learn: 0.0595709	total: 31.7s	remaining: 713ms
978:	learn: 0.0595452	total: 31.7s	remaining: 681ms
979:	learn: 0.0594796	total: 31.8s	remaining: 648ms
980:	learn: 0.0594477	total: 31.8s	remaining: 616ms
981:	learn: 0.0594121	total: 31.8s	remaining: 583ms
982:	learn: 0.0593686	total: 31.8s	remaining: 551ms
983:	learn: 0.0592546	total: 31.9s	remaining: 518ms
984:	learn: 0.0592009	total: 31.9s	remaining: 486ms
985:	learn: 0.0591439	total: 31.9s	remaining: 453ms
986:	learn: 0.0591113	total: 32s	remaining: 421ms
987:	learn: 0.0590606	total: 32s	remaining: 389ms
988:	learn: 0.0589762	total: 32s	remaining: 356ms
989:	learn: 0.0589

,Model,Accuracy,Bal Acc.,Time
1,Random Forest,55.907173,53.734390,1.787995
2,LightGBM,44.514768,43.143480,2.615746
3,CatBoost,44.444444,43.331410,33.254945
4,Extra Trees,42.616034,40.975618,1.117961
5,XGBoost,42.123769,40.864268,7.383671
6,Skl HistGBM,40.154712,39.153528,11.933798
7,Skl GBM,40.014065,39.065228,14.475990
8,AdaBoost,39.873418,35.004487,1.239123
9,Decision Tree,37.482419,36.184777,0.121049


In [30]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[  0 246  11  53  22]
 [  1 136   1   7   0]
 [  1 155  99   7  40]
 [  2 176   4   8   0]
 [  0  32  30   2 389]]
              precision    recall  f1-score   support

         Bus       0.00      0.00      0.00       332
         Car       0.18      0.94      0.31       145
       Still       0.68      0.33      0.44       302
       Train       0.10      0.04      0.06       190
     Walking       0.86      0.86      0.86       453

    accuracy                           0.44      1422
   macro avg       0.37      0.43      0.33      1422
weighted avg       0.45      0.44      0.41      1422

